In [ ]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

2.3.0+cu121
0.18.0+cu121


In [ ]:
device =  "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
import os
# goining modular files  --> modules/(rest files in them)
os.makedirs("modules")

In [ ]:
%%writefile modules/helper_function.py

# imports:
from pathlib import Path
from zipfile import ZipFile
import os
import requests
import torch
import random
from torch import nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def set_seeds(seed:int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

# function to walk through the directory
def walk_through_dir(dir_path):
    for dirpath, dirnames, filenames in os.walk(dir_path):
        print(f"There are {len(dirnames)} directories and {len(filenames)} files in '{dirpath}'.")


# plot decision boundary
def plot_decision_boundary(model: torch.nn.Module,
                           X: torch.Tensor,
                           y: torch.Tensor):
    # put everything to the cpu
    model.to("cpu")

    X, y = X.to("cpu"), y.to("cpu")
    # setup prediction boundary and grids
    x_min, x_max = X[:,0].min() - 0.1, X[:,0].max() + 0.1
    y_min, y_max = X[:,1].min() - 0.1, X[:,1].max() + 0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 101), np.linspace(y_min, y_max, 101))

    # make features
    # np ravel -> flattens the tensor (any dimenstion)
    X_to_pred_on = torch.from_numpy(np.column_stack((xx.ravel(), yy.ravel()))).float()

    # making predictions
    model.eval()
    with torch.inference_mode():
        y_logits = model(X_to_pred_on)
    # test for multiclass or binary and adjusting logits to prediction labels
    if len(torch.unique(y)) > 2:
        y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1) # mutliclass
    else:
        y_pred = torch.round(torch.sigmoid(y_logits)) # binary

    # reshape pred plots
    y_pred = y_pred.reshape(xx.shape).detach().numpy()
    plt.contourf(xx, yy, y_pred, cmap=plt.cm.RdYlBu, alpha=0.7)
    plt.scatter(X[:,0], X[:,1], c=y, s=40, cmap=plt.cm.RdYlBu)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())

# Plot linear data or training and test and predictions
def plot_prediction(
        train_data, train_labels, test_data, test_labels, predictions=None):
    plt.figure(figsize=(10,7))
    # plot the training data in blue
    plt.scatter(train_data, train_labels, c='b', s=4, label='Training data')
    # plot the test data in green
    plt.scatter(test_data, test_labels, c='g', s=4, label='Testing data')

    if predictions is not None:
        # Plot the predictions in red (predictions were made on the test data)
        plt.scatter(test_data, predictions, c='r', s=4, label='Predictions')
    # show legend
    plt.legend(prop={'size': 14})

# calculate accuracy
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

# function for printing train time
def print_train_time(start, end, device=None):
    total_time = end-start
    print(f"\nTrain time on device {device}: {total_time:.3f} seconds")

# plot loss curve
def plot_loss_curves(results):
    accuracy = results["train_acc"]
    test_loss = results["test_loss"]
    loss = results["train_loss"]
    test_accuracy = results["test_accuracy"]

    epochs = range(len(results["train_loss"]))

    plt.figure(figsize=(15,7))
    # plot loss
    plt.subplot(1,2,1)
    plt.plot(epochs, loss, label="loss")
    plt.plot(epochs, test_loss, label="test_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()


    # plot accuracy
    plt.subplot(1,2,2)
    plt.plot(epochs, accuracy, label="train_accuracy")
    plt.plot(epochs, test_accuracy, label="test_accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()


# pred and plot image
from typing import List
import torchvision

def pred_and_plot_image(model:torch.nn.Module,
                        image_path:str,
                        class_names: List[str] = None,
                        transform=None,
                        device: torch.device = "cuda" if torch.cuda.is_available() else "cpu"):
    # 1. load image and convert it to the tensor values (float32)
    target_image =torchvision.io.read_image(str(image_path)).type(torch.float32)

    # 2.divide the image pixel values by 255 to get them b/1 (0,1)
    target_image = target_image / 225.0

    # 3. trasform if necessary
    if transform:
        target_image = transform(target_image)

    # set up the model to the targer device
    model.to(device)

    # put the model to eval mode
    model.eval()

    with torch.inference_mode():
        # add an extra dim to the image
        target_image = target_image.unsqueeze(dim=0)

        # make pred on the image
        target_image_pred = model(target_image.to(device))

    # Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # Convert prediction probabilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

    # plot the image
    plt.imshow(
        target_image.squeeze().permute(1,2,0)
    )
    if class_names:
        title = f"Pred: {class_names[target_image_pred_label.cpu()]} | Prob: {target_image_pred_probs.max().cpu():.3f}"
    else:
        title = f"Pred: {target_image_pred_label} | Prob: {target_image_pred_probs.max().cpu():.3f}"

    plt.title(title)
    plt.axis(False)

# now download data
def download_data(source:str,
                  destination:str,
                  remove_source:bool=True):
    data_path = Path("data/")
    data_path.mkdir(parents=True, exist_ok=True)
    image_path = data_path / destination

    if image_path.is_dir():
        print(f"{image_path} directory exists.")
    else:
        print(f"[INFO] Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)

        # download data
        target_file = Path(source).name
        with open(data_path / target_file, "wb") as f:
            request = requests.get(source)
            print(f"[INFO] Downloading {target_file} from {source}...")
            f.write(request.content)

        # unzip the data
        with ZipFile(data_path / target_file, "r") as zip_ref:
            print(f"[INFO] Unzipping {target_file}...")
            zip_ref.extractall(image_path)

        # remove the zip file
        if remove_source:
            os.remove(data_path / target_file)

    return image_path

# data loader
def create_dataloader(train_dir:str,
                      test_dir:str,
                      transform: transforms.Compose,
                      batch_size:int,
                      num_workers: int = NUM_WORKERS):
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)

    class_names = train_data.classes

    train_dataloader =DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )
    test_dataloader =DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )
    return train_dataloader, test_dataloader, class_names

# a function from utils.py
# used to save the model to depoly them



def save_model(model:torch.nn.Module,
               target_dir: str,
               model_name: str):
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)

    # create model and save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with .pth or .pt"
    model_save_path = target_dir_path / model_name

    # save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
               f=model_save_path)



Writing modules/helper_function.py


In [ ]:
# import the modules to get acess to all the functions
from modules import helper_function as helper

In [ ]:
# download the 20% data from git
data_20_percent_path = helper.download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                            destination="pizza_steak_sushi_20_percent")
data_20_percent_path

[INFO] Did not find data/pizza_steak_sushi_20_percent directory, creating one...
[INFO] Downloading pizza_steak_sushi_20_percent.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip...
[INFO] Unzipping pizza_steak_sushi_20_percent.zip...


PosixPath('data/pizza_steak_sushi_20_percent')

In [ ]:
# setup the train and test dir
train_dir = data_20_percent_path / "train"
test_dir = data_20_percent_path / "test"

In [ ]:
import pathlib

def count_jpg_files(train_dir, test_dir):
    # Ensure the paths are pathlib.Path objects
    train_dir = pathlib.Path(train_dir)
    test_dir = pathlib.Path(test_dir)

    # Count .jpg files in the train directory
    train_jpg_files = list(train_dir.rglob('*.jpg'))
    train_jpg_count = len(train_jpg_files)

    # Count .jpg files in the test directory
    test_jpg_files = list(test_dir.rglob('*.jpg'))
    test_jpg_count = len(test_jpg_files)

    return train_jpg_count, test_jpg_count

train_jpg_count, test_jpg_count = count_jpg_files(train_dir, test_dir)
print(f"Number of .jpg files in the train directory: {train_jpg_count}")
print(f"Number of .jpg files in the test directory: {test_jpg_count}")

Number of .jpg files in the train directory: 450
Number of .jpg files in the test directory: 150


In [ ]:
# we create EffNetB2 feature extractor
# set up the pretrained weights
effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
# transform
effnetb2_transform = effnetb2_weights.transforms()
# set the pretrained model
effnetb2 = torchvision.models.efficientnet_b2(weights=effnetb2_weights)

# freeze the base layer, we will change head layer later
for param in effnetb2.parameters():
    param.requires_grad = False


Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 70.0MB/s]


In [ ]:
effnetb2.classifier


Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1408, out_features=1000, bias=True)
)

In [ ]:
# update the classifier head
from torch import nn
effnetb2.classifier = nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(in_features=1408, out_features=3)
)

In [ ]:
# creating a function to make an Efficientnetb2 classifier
def create_effnetb2_model(num_classes:int=3,
                          seed:int=3):
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    transform = weights.transforms()
    model = torchvision.models.efficientnet_b2(weights=weights)

    # freeze the base layer
    for param in model.parameters():
        param.requires_grad = False

    # change the classifier head to the requirement
    torch.manual_seed(seed)
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3, inplace=True),
        nn.Linear(in_features=1408, out_features=num_classes)
    )
    return model, transform



In [ ]:
effnetb2, effnetb2_transform = create_effnetb2_model()

In [ ]:
# create DataLoader for effnetb2
train_dataloader_effnetb2, test_dataloader_effnetb2, class_names = helper.create_dataloader(train_dir=train_dir,
                                                                                            test_dir=test_dir,
                                                                                            transform=effnetb2_transform,
                                                                                            batch_size=32)

In [ ]:
%%writefile modules/engine.py
import os
import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple
from torch.utils.data import DataLoader
import torchvision
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

def train_step(model:torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    model.train()
    train_loss, train_acc = 0, 0
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item() / len(y_pred)
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model:torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    model.eval()
    test_loss, test_acc = 0, 0
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        test_pred_logits = model(x)
        loss = loss_fn(test_pred_logits, y)
        test_loss += loss.item()
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model:torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []}
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                         dataloader=train_dataloader,
                                         loss_fn=loss_fn,
                                         optimizer=optimizer,
                                         device=device)
        test_loss, test_acc = test_step(model=model,
                                       dataloader=test_dataloader,
                                       loss_fn=loss_fn
                                       device=device)
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        ### New: Experiment tracking ###
        ### New: Use the writer parameter to track experiments ###
        # See if there's a writer, if so, log to it
        if writer:
            # Add results to SummaryWriter
            writer.add_scalars(main_tag="Loss",
                               tag_scalar_dict={"train_loss": train_loss,
                                                "test_loss": test_loss},
                               global_step=epoch)
            writer.add_scalars(main_tag="Accuracy",
                               tag_scalar_dict={"train_acc": train_acc,
                                                "test_acc": test_acc},
                               global_step=epoch)

            # Close the writer
            writer.close()
        else:
            pass
    return results

Writing modules/engine.py


In [ ]:
from modules import engine

# setting up optimizer
optimizer = torch.optim.Adam(params=effnetb2.parameters(), lr=1e-3)

# loss function
loss = torch.nn.CrossEntropyLoss()
# set seeds
helper.set_seeds()
effnetb2 = effnetb2.to(device)
effnetb2_results = engine.train(model=effnetb2,
                                train_dataloader=train_dataloader_effnetb2,
                                test_dataloader=test_dataloader_effnetb2,
                                loss_fn=loss,
                                optimizer=optimizer,
                                epochs=10,
                                device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch: 1 | train_loss: 0.9556 | train_acc: 0.5667 | test_loss: 0.7378 | test_acc: 0.9136
Epoch: 2 | train_loss: 0.7036 | train_acc: 0.8458 | test_loss: 0.5996 | test_acc: 0.9074
Epoch: 3 | train_loss: 0.5809 | train_acc: 0.8896 | test_loss: 0.4967 | test_acc: 0.9472
Epoch: 4 | train_loss: 0.4456 | train_acc: 0.9125 | test_loss: 0.4369 | test_acc: 0.9409
Epoch: 5 | train_loss: 0.4194 | train_acc: 0.8938 | test_loss: 0.4015 | test_acc: 0.9261
Epoch: 6 | train_loss: 0.4416 | train_acc: 0.8875 | test_loss: 0.3583 | test_acc: 0.9534
Epoch: 7 | train_loss: 0.4258 | train_acc: 0.8354 | test_loss: 0.3318 | test_acc: 0.9472
Epoch: 8 | train_loss: 0.3922 | train_acc: 0.8917 | test_loss: 0.3526 | test_acc: 0.9199
Epoch: 9 | train_loss: 0.3689 | train_acc: 0.8938 | test_loss: 0.3169 | test_acc: 0.9074
Epoch: 10 | train_loss: 0.3660 | train_acc: 0.8688 | test_loss: 0.2849 | test_acc: 0.9597


In [ ]:
# saving the effnetb2 feature extractor
helper.save_model(model=effnetb2,
                  target_dir="models",
                  model_name="pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth")

[INFO] Saving model to: models/pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth


In [ ]:
from pathlib import Path

# Get the model size in bytes then convert to megabytes
pretrained_effnetb2_model_size = Path("models/pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth").stat().st_size // (1024*1024) # division converts bytes to megabytes (roughly)
print(f"Pretrained EffNetB2 feature extractor model size: {pretrained_effnetb2_model_size} MB")


Pretrained EffNetB2 feature extractor model size: 29 MB


In [ ]:
effnetb2_total_params = sum(torch.numel(p) for p in effnetb2.parameters())
effnetb2_total_params

7705221

In [ ]:
# Create a dictionary with EffNetB2 statistics
effnetb2_stats = {"test_loss": effnetb2_results["test_loss"][-1],
                  "test_acc": effnetb2_results["test_acc"][-1],
                  "number_of_parameters": effnetb2_total_params,
                  "model_size (MB)": pretrained_effnetb2_model_size}
effnetb2_stats

{'test_loss': 0.28485937118530275,
 'test_acc': 0.959659090909091,
 'number_of_parameters': 7705221,
 'model_size (MB)': 29}

In [ ]:
# Now we create pretrained vit feature extractor
vit = torchvision.models.vit_b_16()
vit.heads


Sequential(
  (head): Linear(in_features=768, out_features=1000, bias=True)
)

In [ ]:
# feature extractor for vit_base
def create_vit_model(num_classes: int=3,
                     seed:int=42):
    weights = torchvision.models.ViT_B_16_Weights.DEFAULT
    transform = weights.transforms()
    model = torchvision.models.vit_b_16(weights=weights)
    # freeze all the layers except base
    for param in model.parameters():
        param.requires_grad = False

    # change the classifier head
    torch.manual_seed(seed)
    model.heads = nn.Sequential(
        nn.Linear(in_features=768,
                  out_features=num_classes)
    )
    return model, transform

In [ ]:
vit, vit_transform = create_vit_model()

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:02<00:00, 116MB/s]


In [ ]:
# set up dataloader
train_dataloader_vit, test_dataloader_vit, class_names = helper.create_dataloader(train_dir=train_dir,
                                                                                  test_dir=test_dir,
                                                                                  transform=vit_transform,
                                                                                  batch_size=32)

In [ ]:
# now training vit feature extractor

# set up optimizer
optimizer = torch.optim.Adam(params=vit.parameters(), lr=1e-3)
# set up loss function
loss_fn = torch.nn.CrossEntropyLoss()

helper.set_seeds()
vit = vit.to(device)
vit_results = engine.train(model=vit,
                           train_dataloader=train_dataloader_vit,
                           test_dataloader=test_dataloader_vit,
                           loss_fn=loss_fn,
                           optimizer=optimizer,
                           epochs=10,
                           device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch: 1 | train_loss: 0.7020 | train_acc: 0.7521 | test_loss: 0.2714 | test_acc: 0.9381
Epoch: 2 | train_loss: 0.2532 | train_acc: 0.9062 | test_loss: 0.1672 | test_acc: 0.9602
Epoch: 3 | train_loss: 0.1764 | train_acc: 0.9542 | test_loss: 0.1273 | test_acc: 0.9693
Epoch: 4 | train_loss: 0.1276 | train_acc: 0.9625 | test_loss: 0.1074 | test_acc: 0.9722
Epoch: 5 | train_loss: 0.1159 | train_acc: 0.9646 | test_loss: 0.0953 | test_acc: 0.9784
Epoch: 6 | train_loss: 0.1274 | train_acc: 0.9375 | test_loss: 0.0832 | test_acc: 0.9722
Epoch: 7 | train_loss: 0.0897 | train_acc: 0.9771 | test_loss: 0.0845 | test_acc: 0.9784
Epoch: 8 | train_loss: 0.0919 | train_acc: 0.9812 | test_loss: 0.0764 | test_acc: 0.9722
Epoch: 9 | train_loss: 0.0922 | train_acc: 0.9792 | test_loss: 0.0734 | test_acc: 0.9784
Epoch: 10 | train_loss: 0.0658 | train_acc: 0.9833 | test_loss: 0.0644 | test_acc: 0.9847


In [ ]:
helper.save_model(model=vit,
                  target_dir="models",
                  model_name="pretrained_vit_feature_extractor_pizza_steak_sushi_20_percent.pth")

[INFO] Saving model to: models/pretrained_vit_feature_extractor_pizza_steak_sushi_20_percent.pth


In [ ]:
from pathlib import Path

# Get the model size in bytes then convert to megabytes
pretrained_vit_model_size = Path("models/pretrained_vit_feature_extractor_pizza_steak_sushi_20_percent.pth").stat().st_size // (1024*1024) # division converts bytes to megabytes (roughly)
print(f"Pretrained ViT feature extractor model size: {pretrained_vit_model_size} MB")

Pretrained ViT feature extractor model size: 327 MB


In [ ]:
# collecting stats
vit_total_parameters = sum(torch.numel(p) for p in vit.parameters())
vit_total_parameters

85800963

In [ ]:
# Create ViT statistics dictionary
vit_stats = {"test_loss": vit_results["test_loss"][-1],
             "test_acc": vit_results["test_acc"][-1],
             "number_of_parameters": vit_total_parameters,
             "model_size (MB)": pretrained_vit_model_size}

vit_stats

{'test_loss': 0.06443451717495918,
 'test_acc': 0.984659090909091,
 'number_of_parameters': 85800963,
 'model_size (MB)': 327}

In [ ]:
# get a"ll test data paths for timing the model performance
print(f"[INFO] Finding all the file path with '.jpg' directory: {test_dir}")
test_data_paths = list(Path(test_dir).glob("*/*.jpg"))
test_data_paths[:5]


[INFO] Finding all the file path with '.jpg' directory: data/pizza_steak_sushi_20_percent/test


[PosixPath('data/pizza_steak_sushi_20_percent/test/pizza/419962.jpg'),
 PosixPath('data/pizza_steak_sushi_20_percent/test/pizza/1315645.jpg'),
 PosixPath('data/pizza_steak_sushi_20_percent/test/pizza/2250611.jpg'),
 PosixPath('data/pizza_steak_sushi_20_percent/test/pizza/930553.jpg'),
 PosixPath('data/pizza_steak_sushi_20_percent/test/pizza/2398925.jpg')]

In [ ]:
# a function to make predictions across the test dataset
import pathlib
import torch
from PIL import Image
from timeit import default_timer as timer
from tqdm.auto import tqdm
from typing import List, Dict

# 1. creating the function
def pred_and_store(paths:List[pathlib.Path],
                   model:torch.nn.Module,
                   transform:torchvision.transforms,
                   class_names:List[str],
                   device: str="cuda" if torch.cuda.is_available() else "cpu") -> List[Dict]:
    # create a empty list
    pred_list = []
    # looping through target path
    for path in tqdm(paths):
        # dict to store pred for all the sample
        pred_dict = {}

        # get sample path
        pred_dict["image_path"] = path
        class_name = path.parent.stem
        pred_dict["class_name"] = class_name
        # start pred timer
        start_time = timer()

        img = Image.open(path)
        # transform the image
        transformed_image = transform(img).unsqueeze(0).to(device)
        # prepare the model for sending to the device
        model.to(device)
        model.eval()

        # getting the prediction
        with torch.inference_mode():
            pred_logit = model(transformed_image) # perform inference on target sample
            pred_prob = torch.softmax(pred_logit, dim=1) # turn logits into prediction probabilities
            pred_label = torch.argmax(pred_prob, dim=1) # turn prediction probabilities into prediction label
            pred_class = class_names[pred_label.cpu()] # hardcode prediction class to be on CPU

            # putting dictonary on the cpu
            pred_dict["pred_prob"] = round(pred_prob.unsqueeze(0).max().cpu().item(), 4)
            pred_dict["pred_class"] = pred_class

            end_time = timer()
            pred_dict["time_for_pred"] = round(end_time-start_time, 4)
        # is the pred correct or no
        pred_dict["correct"] = class_name == pred_class
        # add dict to the list
        pred_list.append(pred_dict)
    return pred_list

In [ ]:
# Make predictions across test dataset with EffNetB2
effnetb2_test_pred_dicts = pred_and_store(paths=test_data_paths,
                                          model=effnetb2,
                                          transform=effnetb2_transform,
                                          class_names=class_names,
                                          device="cpu") # make predictions on CPU

  0%|          | 0/150 [00:00<?, ?it/s]

In [ ]:
effnetb2_test_pred_dicts[:2]

[{'image_path': PosixPath('data/pizza_steak_sushi_20_percent/test/pizza/419962.jpg'),
  'class_name': 'pizza',
  'pred_prob': 0.3517,
  'pred_class': 'sushi',
  'time_for_pred': 0.2669,
  'correct': False},
 {'image_path': PosixPath('data/pizza_steak_sushi_20_percent/test/pizza/1315645.jpg'),
  'class_name': 'pizza',
  'pred_prob': 0.7393,
  'pred_class': 'pizza',
  'time_for_pred': 0.0917,
  'correct': True}]

In [ ]:
# creating it into a dataframes
import pandas as pd
effnetb2_test_pred_df = pd.DataFrame(effnetb2_test_pred_dicts)
effnetb2_test_pred_df.head()

,image_path,class_name,pred_prob,pred_class,time_for_pred,correct
0,data/pizza_steak_sushi_20_percent/test/pizza/4...,pizza,0.3517,sushi,0.2669,False
1,data/pizza_steak_sushi_20_percent/test/pizza/1...,pizza,0.7393,pizza,0.0917,True
2,data/pizza_steak_sushi_20_percent/test/pizza/2...,pizza,0.8663,pizza,0.0912,True
3,data/pizza_steak_sushi_20_percent/test/pizza/9...,pizza,0.9277,pizza,0.0859,True
4,data/pizza_steak_sushi_20_percent/test/pizza/2...,pizza,0.8804,pizza,0.1009,True


In [ ]:
effnetb2_test_pred_df.correct.value_counts()

correct
True     144
False      6
Name: count, dtype: int64

In [ ]:
effnetb2_average_time_per_pred = round(effnetb2_test_pred_df.time_for_pred.mean(), 4)
print(f"Avg time for effnetb2: {effnetb2_average_time_per_pred} seconds")

Avg time for effnetb2: 0.1007 seconds


In [ ]:
# Add EffNetB2 average prediction time to stats dictionary
effnetb2_stats["time_per_pred_cpu"] = effnetb2_average_time_per_pred
effnetb2_stats

{'test_loss': 0.28485937118530275,
 'test_acc': 0.959659090909091,
 'number_of_parameters': 7705221,
 'model_size (MB)': 29,
 'time_per_pred_cpu': 0.1007}

In [ ]:
# now the same thing for vit model
vit_test_pred_dicts = pred_and_store(paths=test_data_paths,
                               model=vit,
                               transform=vit_transform,
                               class_names=class_names,
                               device="cpu")

  0%|          | 0/150 [00:00<?, ?it/s]

In [ ]:
# checking out the results
vit_test_pred_dicts[:2]

[{'image_path': PosixPath('data/pizza_steak_sushi_20_percent/test/pizza/419962.jpg'),
  'class_name': 'pizza',
  'pred_prob': 0.9977,
  'pred_class': 'pizza',
  'time_for_pred': 0.6002,
  'correct': True},
 {'image_path': PosixPath('data/pizza_steak_sushi_20_percent/test/pizza/1315645.jpg'),
  'class_name': 'pizza',
  'pred_prob': 0.9698,
  'pred_class': 'pizza',
  'time_for_pred': 0.3717,
  'correct': True}]

In [ ]:
# turning it to a dataframe
vit_test_pred_df = pd.DataFrame(vit_test_pred_dicts)
vit_test_pred_df.head()

,image_path,class_name,pred_prob,pred_class,time_for_pred,correct
0,data/pizza_steak_sushi_20_percent/test/pizza/4...,pizza,0.9977,pizza,0.6002,True
1,data/pizza_steak_sushi_20_percent/test/pizza/1...,pizza,0.9698,pizza,0.3717,True
2,data/pizza_steak_sushi_20_percent/test/pizza/2...,pizza,0.9986,pizza,0.3694,True
3,data/pizza_steak_sushi_20_percent/test/pizza/9...,pizza,0.9982,pizza,0.3976,True
4,data/pizza_steak_sushi_20_percent/test/pizza/2...,pizza,0.9982,pizza,0.3729,True


In [ ]:
vit_test_pred_df.correct.value_counts()

correct
True     148
False      2
Name: count, dtype: int64

In [ ]:
# calculating the average time
vit_average_time_per_pred = round(vit_test_pred_df.time_for_pred.mean(), 4)
print(f"Avg time for vit: {vit_average_time_per_pred} seconds")

Avg time for vit: 0.421 seconds


In [ ]:
# Add average prediction time for ViT model on CPU
vit_stats["time_per_pred_cpu"] = vit_average_time_per_pred
vit_stats

{'test_loss': 0.06443451717495918,
 'test_acc': 0.984659090909091,
 'number_of_parameters': 85800963,
 'model_size (MB)': 327,
 'time_per_pred_cpu': 0.421}

In [ ]:
# comparing the model and results
df = pd.DataFrame([effnetb2_stats, vit_stats])
df["model"] = ["Effnetb2", "vit"]
df["test_acc"] = round(df["test_acc"] * 100, 2)
df

,test_loss,test_acc,number_of_parameters,model_size (MB),time_per_pred_cpu,model
0,0.284859,95.97,7705221,29,0.1007,Effnetb2
1,0.064435,98.47,85800963,327,0.4210,vit


In [ ]:
# Compare ViT to EffNetB2 across different characteristics
pd.DataFrame(data=(df.set_index("model").loc["vit"] / df.set_index("model").loc["Effnetb2"]), # divide ViT statistics by EffNetB2 statistics
             columns=["ViT to EffNetB2 ratios"]).T

,test_loss,test_acc,number_of_parameters,model_size (MB),time_per_pred_cpu
ViT to EffNetB2 ratios,0.226198,1.02605,11.135432,11.275862,4.180735


In [ ]:
# creating a function to map out input and output
# what it does?
# input: image -> transform -> predict with EffNetB2 -> output: pred, pred prob, time taken
from typing import Tuple, Dict
def predict(img) -> Tuple[Dict, float]:
    # start the timer
    start_time = timer()
    # transform image
    img = effnetb2_transform(img).unsqueeze(0)
    # put the model to eval mode
    effnetb2.eval()
    with torch.inference_mode():
        # predict
        pred_probs = torch.softmax(effnetb2(img), dim=1)
    # creating prediction labels
    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
    # calculating prediction time
    pred_time = round(timer() - start_time, 5)

    return pred_labels_and_probs, pred_time

In [ ]:
# testing the function
import random
test_data_path = list(Path(test_dir).glob("*/*.jpg"))

# randomly select an image
random_image_path = random.sample(test_data_path, k=1)[0]
# open the target image
image = Image.open(random_image_path)
print(f"[INFO] Predicting on image at path: {random_image_path}\n")
pred_dict, pred_time = predict(img=image)
print(f"Prediction label and probability dictionary: \n{pred_dict}")
print(f"Prediction time: {pred_time} seconds")

[INFO] Predicting on image at path: data/pizza_steak_sushi_20_percent/test/steak/3622237.jpg

Prediction label and probability dictionary: 
{'pizza': 0.042616166174411774, 'steak': 0.9375132322311401, 'sushi': 0.01987054944038391}
Prediction time: 0.48192 seconds


In [ ]:
# creating a list of examples for gardio to act as an input
example_list = [[str(filepath)] for filepath in random.sample(test_data_paths, k=6)]
example_list

[['data/pizza_steak_sushi_20_percent/test/pizza/1315645.jpg'],
 ['data/pizza_steak_sushi_20_percent/test/steak/3497585.jpg'],
 ['data/pizza_steak_sushi_20_percent/test/sushi/2378406.jpg'],
 ['data/pizza_steak_sushi_20_percent/test/steak/108310.jpg'],
 ['data/pizza_steak_sushi_20_percent/test/steak/367422.jpg'],
 ['data/pizza_steak_sushi_20_percent/test/sushi/1346344.jpg']]

In [ ]:
!pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 9.

In [ ]:
import gradio as gr
# create tile, description and strings
title = "FoodVision Mini"
description = "An EfficientNetB2 feature extractor computer vision model to classify images of food as pizza, steak or sushi."
article = "Follow up by Dipanshu Singh (https://www.learnpytorch.io/09_pytorch_model_deployment/). "

# create a demo
demo = gr.Interface(fn=predict,
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=3, label="predictions"),
                             gr.Number(label="Prediction time")],
                             examples=example_list,
                    title=title,
                    description=description,
                    article=article)

demo.launch(debug=True,share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f91c154f4153ba97c7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f91c154f4153ba97c7.gradio.live


In [ ]:
import shutil
from pathlib import Path

# create a mini demo path
foodversion_mini_demo_path = Path("demos/foodversion_mini")

if foodversion_mini_demo_path.exists():
    shutil.rmtree(foodversion_mini_demo_path)
    foodversion_mini_demo_path.mkdir(parents=True, exist_ok=True)
else:
    foodversion_mini_demo_path.mkdir(parents=True, exist_ok=True)

!ls demos/foodversion_mini/

In [ ]:
# creating examples for the app

foodversion_mini_examples_path = foodversion_mini_demo_path / "examples"
foodversion_mini_examples_path.mkdir(parents=True, exist_ok=True)

# collect random test data images
foodversion_mini_examples = [Path('data/pizza_steak_sushi_20_percent/test/pizza/1315645.jpg'),
                             Path('data/pizza_steak_sushi_20_percent/test/steak/3497585.jpg'),
                             Path('data/pizza_steak_sushi_20_percent/test/sushi/2378406.jpg'),
                             Path('data/pizza_steak_sushi_20_percent/test/steak/108310.jpg'),
                             Path('data/pizza_steak_sushi_20_percent/test/steak/367422.jpg'),
                             Path('data/pizza_steak_sushi_20_percent/test/sushi/1346344.jpg')]

# copy it to the example dirs
for example in foodversion_mini_examples:
    destination = foodversion_mini_examples_path / example.name
    print(f"[INFO] Copying {example} to {destination}")
    shutil.copy2(src=example, dst=destination)

[INFO] Copying data/pizza_steak_sushi_20_percent/test/pizza/1315645.jpg to demos/foodversion_mini/examples/1315645.jpg
[INFO] Copying data/pizza_steak_sushi_20_percent/test/steak/3497585.jpg to demos/foodversion_mini/examples/3497585.jpg
[INFO] Copying data/pizza_steak_sushi_20_percent/test/sushi/2378406.jpg to demos/foodversion_mini/examples/2378406.jpg
[INFO] Copying data/pizza_steak_sushi_20_percent/test/steak/108310.jpg to demos/foodversion_mini/examples/108310.jpg
[INFO] Copying data/pizza_steak_sushi_20_percent/test/steak/367422.jpg to demos/foodversion_mini/examples/367422.jpg
[INFO] Copying data/pizza_steak_sushi_20_percent/test/sushi/1346344.jpg to demos/foodversion_mini/examples/1346344.jpg


In [ ]:
# checking if the examples are actually present in the file
example_list = [["examples/" + example] for example in os.listdir(foodversion_mini_examples_path)]
example_list

[['examples/1315645.jpg'],
 ['examples/1346344.jpg'],
 ['examples/2378406.jpg'],
 ['examples/367422.jpg'],
 ['examples/108310.jpg'],
 ['examples/3497585.jpg']]

In [ ]:
# source for out target model
effnetb2_foodversion_mini_model_path = "models/pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth"
# creating a destination path
effnetb2_foodversion_mini_model_destination = foodversion_mini_demo_path / effnetb2_foodversion_mini_model_path.split("/")[1]

# moving the files

try:

    # move the model
    shutil.move(src=effnetb2_foodversion_mini_model_path,
                dst=effnetb2_foodversion_mini_model_destination)
    print(f"[INFO] Model move complete.")
except:
     print(f"[INFO] No model found at {effnetb2_foodversion_mini_model_path}, perhaps its already been moved?")
     print(f"[INFO] Model exists at {effnetb2_foodversion_mini_model_destination}: {effnetb2_foodversion_mini_model_destination.exists()}")

[INFO] Model move complete.


In [ ]:
%%writefile demos/foodversion_mini/model.py

import torch
import torch.nn as nn
import torchvision

def create_effnetb2_model(num_classes: int=3,
                           seed:int=42):
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    transform = weights.transforms()
    model = torchvision.models.efficientnet_b2(weights=weights)
    # freeze all the layers except base
    for param in model.parameters():
        param.requires_grad = False
    # change the classifier head
    torch.manual_seed(seed)
    model.heads = nn.Sequential(
        nn.Dropout(p=0.3, inplace=True),
        nn.Linear(in_features=1480,
                  out_features=num_classes)
    )
    return model, transform

Overwriting demos/foodversion_mini/model.py


In [ ]:
%%writefile demos/foodversion_mini/app.py

import os
import torch
import gradio
from model import create_effnetb2_model
from timeit import default_timer as timer
from typing import Tuple, Dict

# set up class names
class_names = ["pizza", "steak", "sushi"]
# model transformation and prep
effnetb2, effnetb2_transform = create_effnetb2_model()
# loading saved weights to cpu
effnetb2.load_state_dict(
    torch.load(f"models/pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth",
               map_location=torch.device("cpu"))
)

# prediction function
def predict(img) -> Tuple[Dict, float]:
    start_time = timer()

    img = effnetb2_transform(img).unsqueeze(0)
    effnetb2.eval()
    with torch.inference_mode():
        pred_probs = torch.softmax(effnetb2(img), dim=1)

    # creating pred labels
    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
    pred_time = round(timer() - start_time, 5)
    return pred_labels_and_probs, pred_time

# creating the app
title = "FoodVision Mini"
description = "An EfficientNetB2 feature extractor computer vision model to classify images of food as pizza, steak or sushi."
article = "Follow up by Dipanshu Singh (https://www.learnpytorch.io/09_pytorch_model_deployment/). "


# Create examples list from "examples/" directory
example_list = [["examples/" + example] for example in os.listdir("examples")]

# create a demo
demo = gr.Interface(fn=predict,
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=3, label="predictions"),
                             gr.Number(label="Prediction time")],
                             examples=example_list,
                    title=title,
                    description=description,
                    article=article)

demo.launch(debug=True,share=True)

Overwriting demos/foodversion_mini/app.py


In [ ]:
%%writefile demos/foodversion_mini/requirements.txt
torch==1.12.0
torchvision==0.13.0
gradio==3.1.4

Overwriting demos/foodversion_mini/requirements.txt


In [ ]:
!ls demos/foodversion_mini

app.py	  model.py								  requirements.txt
examples  pretrained_effnetb2_feature_extractor_pizza_steak_sushi_20_percent.pth


In [ ]:
!python app.py

python3: can't open file '/content/app.py': [Errno 2] No such file or directory
